# Text to Image generator using Ngrok and streamlit in Google Colab

Presenter: [Zartashia Afzal](https://www.linkedin.com/in/zartashiaafzal/)

[Beginner friendly Article guide for this notebook](https://www.linkedin.com/pulse/build-your-ai-image-generator-streamlit-colab-hugging-zartashia-afzal-xiaqf/?trackingId=SzDbsraiQxaUd1QnY6vlnA%3D%3D)

# 1- Installing libraries

In [1]:

!pip install pyngrok


In [3]:
from pyngrok import ngrok



Go to [Ngrok website](https://ngrok.com/), make account and get auth token

In [4]:
!ngrok authtoken n8bjsncsfnnkGhdbcsdc_jsdunZzSi5chsbcjahdwsczi  # Replace token with the actual token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
!pip install diffusers torch torchvision streamlit  numpy huggingface-hub

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [6]:
# !pip install -r /content/requirements_clean.txt  (In case if you want to install all libraries from requirement.txt instead of above code in notebook)


In [8]:
# !pip install pillow==10.4.0


# 2-Streamlit .py script

In [9]:
%%writefile app_1.py
import streamlit as st
from diffusers import DiffusionPipeline
import torch
from PIL import Image
import io
import numpy as np
import time

@st.cache_resource
def load_model():
    model = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
    model.to("cuda" if torch.cuda.is_available() else "cpu")
    return model

generator = load_model()

st.title('Text to Image Generator')
prompt = st.text_input("Enter your prompt:", "A scenic landscape painting")

if st.button('Generate Image'):
    start_time = time.time()  # Start timing
    with st.spinner('Generating image...'):
        # Generate the image with a low number of inference steps
        output = generator(prompt, num_inference_steps=5, eta=0.0)  # Adjust ETA to possibly reduce computation
        image = output.images[0]

        # Convert to NumPy array
        image = np.array(image)

        # Convert to PIL Image
        pil_image = Image.fromarray(image.astype(np.uint8))  # Cast to uint8 for PIL compatibility

        # Display the image
        st.image(pil_image, caption='Generated Image', use_column_width=True)

        # Provide a download button
        buf = io.BytesIO()
        pil_image.save(buf, format="JPEG")
        byte_im = buf.getvalue()
        st.download_button(
            label="Download Image",
            data=byte_im,
            file_name="generated_image.jpg",
            mime="image/jpeg"
        )

    # Record and display time taken
    end_time = time.time()
    st.write(f"Time taken to generate image: {end_time - start_time:.2f} seconds")

st.write("Enter a textual prompt and press generate to create an image.")


Writing app_1.py


# Launching app

In [10]:
!streamlit run app_1.py &>/dev/null&


In [ ]:
# Start an HTTP tunnel on the default port 8501 for Streamlit
url = ngrok.connect(8501)
print("Access your Streamlit app at:", url)

# you will get url after running it 

In [12]:
# ngrok.disconnect(url)


In [ ]:
active_tunnels = ngrok.get_tunnels()
print(active_tunnels)


In [14]:
# ngrok.kill()
